In [46]:
# Download US power output PV Watts (version 8) from 

In [47]:
# Documentation: https://developer.nrel.gov/docs/api-key/
# https://pvwatts.nrel.gov/downloads/pvwattsv5.pdf

In [1]:
import numpy as np
import scipy as sp
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
hrh_apikey = 'GNoTfD5IZWwIEz24zB5Wn0aEhDvNJSep5bwapzTI'
data_path = '/Users/hrh/Desktop/AI/erdos/rapower_data/'

In [3]:
# 1. Define PVWatts parameters and your API key
parameters = {
    'format': 'json',
    'system_capacity': 1000,
    'module_type': 0,
    'losses': 14.08,
    'array_type': 1,
    'tilt': 40,
    'azimuth': 180,
    'lat': 40.0099983215332,
    'lon':  -105.2600021362305,
    'dataset': 'tmy3',
    'radius': 25,
    'timeframe': 'monthly',
    'api_key': hrh_apikey  # Replace with your API key
}



In [51]:
# url = 'https://developer.nrel.gov/api/pvwatts/v8'

# # 2. Send request
# response = requests.get(url, params=parameters)
# data = response.json()

# # 3. Convert the output data to a DataFrame
# df = pd.DataFrame(data['outputs'])

# print(response)

In [52]:
##### Try to get data for California on a grid with lat_delta and lon_delta resolution
# lat_delta = 0.25
# lon_delta = 0.25

# # #CONUS box
# # top = 49.3457868 # north lat
# # left = -124.7844079 # west long
# # right = -66.9513812 # east long
# # bottom =  24.7433195 # south lat

# #California box
# top    = 42.009518
# bottom = 32.534156
# left   = -124.409591 
# right  =  -114.131211


In [26]:
# 1. Define a list of latitudes and longitudes for CONUS
# lats = list(np.arange(bottom, top, lat_delta))  # from southernmost to northernmost points with a 5-degree step
# lons = list(np.arange(left, right, lon_delta))  # from westernmost to easternmost points with a 5-degree step

all_data = []
##### Read lats and lons CA_grid_dataframe #############
ca_gridpath = '/Users/hrh/Desktop/AI/erdos/SolarFarmPrediction/data_including_land_cover.csv'

df_main             = pd.read_csv(ca_gridpath)
ca_grid             = df_main.loc[df_main['State']=='CA']
ca_grid_usable      = ca_grid.loc[ca_grid['land_cover']==1.0]
ca_coords           = list(zip(ca_grid_usable['Latitude'], ca_grid_usable['Longitude']))
#Check if ca_coords are within california grid box
ca_lats = [item[0] for item in ca_coords] 
ca_lons = [item[1] for item in ca_coords]
(min(ca_lats),max(ca_lats)),(min(ca_lons),max(ca_lons))

((32.61826367503021, 41.98152421968207),
 (-124.36136888868832, -114.14033495079964))

In [31]:
%%time
    for (lat,lon) in ca_coords:
        parameters = {
            'format': 'json',
            'system_capacity': 1000, #1000kW = 1 MW, 1 MW or greater is considered utility-scale
            'module_type': 0,       # 0- Standard module, 1- Permium, 2-Thin film
            'losses': 14,           # Losses in percentage
            'array_type': 0,        # Open Rack: Also known as ground mount.
            'tilt': 40,
            'azimuth': 180,         # This means that the solar array is facing South in the Northeren Hemisphere
            'lat': lat,
            'lon': lon,
            'dataset': 'tmy3',      #tmy2 is 1960-1990, tmy3 is 1990-2005
            'radius': 100,            # 0-Pick the station nearest to the given (lat,lon), 50 -50 miles
            'timeframe': 'monthly',
            'api_key': hrh_apikey
        }

        url = 'https://developer.nrel.gov/api/pvwatts/v8'
        response = requests.get(url, params=parameters)
        data = response.json()
             
        # Extract and append relevant data to the all_data list
        # Extract relevant data
        #print('Processing (lat,lon):',lat,lon,'\n')
        
        # Check if 'outputs' is in the data
        if 'outputs' not in data:
            print(f"No data available for latitude {lat} and longitude {lon}")
            continue

        output_data = data['outputs']
        input_data  = data['inputs']

        output_data['latitude']  = input_data['lat']
        output_data['longitude'] = input_data['lon']
        all_data.append(output_data)
        
        
# Convert all_data to a DataFrame
df = pd.DataFrame(all_data)

print(df)

No data available for latitude 37.8471729 and longitude -122.2715295
No data available for latitude 37.1544497 and longitude -121.5602301
No data available for latitude 38.0898882 and longitude -122.1426351
No data available for latitude 38.4300017 and longitude -122.7255246
No data available for latitude 38.4591205 and longitude -122.7449891
No data available for latitude 38.4888689 and longitude -122.7776581
No data available for latitude 38.453198 and longitude -122.731544
No data available for latitude 38.4271074 and longitude -122.7856681
No data available for latitude 37.9414521 and longitude -122.007725
No data available for latitude 39.3703737 and longitude -123.3024709
No data available for latitude 33.8373954999999 and longitude -117.3716772
No data available for latitude 33.9944764999999 and longitude -117.4297158
No data available for latitude 37.7564202566745 and longitude -122.198718031487
No data available for latitude 37.7027656058429 and longitude -121.950435688049
No 

KeyboardInterrupt: 

In [6]:
df

,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor,latitude,longitude
0,"[58064.64523772797, 95505.9549950513, 121219.7...","[71.28546168778718, 117.7538988225777, 154.245...","[2.299531022186683, 4.205496386520631, 4.97566...","[61352.53824535836, 100296.3468153766, 127095....",1.527470e+06,5.420063,17.436877,37.8471729,-120.2232171
1,"[83052.90909057189, 101904.1185675195, 130576....","[100.8174029877149, 125.6366497032874, 162.857...","[3.252174289926287, 4.487023203688837, 5.25345...","[87256.4486042592, 106918.7489756171, 136914.7...",1.503490e+06,5.156304,17.163124,37.1544497,-123.2869629
2,"[64333.53765837142, 79710.70713752598, 96341.6...","[79.37921239410372, 98.29785315657168, 119.565...","[2.560619754648507, 3.510637612734703, 3.85694...","[67945.47351754847, 83895.0453953961, 101405.8...",1.497993e+06,5.235499,17.100375,38.0898882,-122.0847692
3,"[77990.5358581602, 90938.93051785043, 135695.8...","[94.09218583519545, 111.8041508816429, 170.460...","[3.035231801135337, 3.993005388630105, 5.49873...","[81971.77795004952, 95608.01000357303, 142093....",1.519804e+06,5.259745,17.349356,38.4300017,-122.2715295
4,"[71187.68931894137, 91325.47762727663, 126537....","[87.86079743893579, 112.648830751238, 158.8739...","[2.834219272223735, 4.02317252682993, 5.124965...","[75096.34961384782, 95950.40507552765, 132856....",1.522243e+06,5.380545,17.377201,38.4591205,-121.5602301
...,...,...,...,...,...,...,...,...,...
995,"[107265.182354745, 134276.019867511, 128014.07...","[124.1384581270522, 158.5507983603165, 156.865...","[4.004466391195231, 5.662528512868448, 5.06018...","[112428.6481571912, 140902.6655321975, 134651....",1.459818e+06,5.027627,16.664588,39.22182637494257,-97.6888179075072
996,"[107265.182354745, 134276.019867511, 128014.07...","[124.1384581270522, 158.5507983603165, 156.865...","[4.004466391195231, 5.662528512868448, 5.06018...","[112428.6481571912, 140902.6655321975, 134651....",1.459818e+06,5.027627,16.664588,39.3203870122547,-97.7638646004839
997,"[107265.182354745, 134276.019867511, 128014.07...","[124.1384581270522, 158.5507983603165, 156.865...","[4.004466391195231, 5.662528512868448, 5.06018...","[112428.6481571912, 140902.6655321975, 134651....",1.459818e+06,5.027627,16.664588,39.418947649566825,-97.8209994029457
998,"[92789.40788623082, 88502.69848567061, 147788....","[110.863837704466, 106.2788310789551, 185.1160...","[3.576252829176322, 3.795672538534109, 5.97148...","[97488.8026381655, 93219.84003819182, 155050.4...",1.487156e+06,5.235474,16.976670,39.51750828687895,-112.266637978604


In [8]:
df.to_csv('pvwatts_monthly_ca_1990_2005.csv',index=False)

In [9]:
df1 = pd.read_csv('pvwatts_monthly_ca_1990_2005.csv')
df1

,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor,latitude,longitude
0,"[58064.64523772797, 95505.9549950513, 121219.7...","[71.28546168778718, 117.7538988225777, 154.245...","[2.299531022186683, 4.205496386520631, 4.97566...","[61352.53824535836, 100296.3468153766, 127095....",1.527470e+06,5.420063,17.436877,37.847173,-120.223217
1,"[83052.90909057189, 101904.1185675195, 130576....","[100.8174029877149, 125.6366497032874, 162.857...","[3.252174289926287, 4.487023203688837, 5.25345...","[87256.4486042592, 106918.7489756171, 136914.7...",1.503490e+06,5.156304,17.163124,37.154450,-123.286963
2,"[64333.53765837142, 79710.70713752598, 96341.6...","[79.37921239410372, 98.29785315657168, 119.565...","[2.560619754648507, 3.510637612734703, 3.85694...","[67945.47351754847, 83895.0453953961, 101405.8...",1.497993e+06,5.235499,17.100375,38.089888,-122.084769
3,"[77990.5358581602, 90938.93051785043, 135695.8...","[94.09218583519545, 111.8041508816429, 170.460...","[3.035231801135337, 3.993005388630105, 5.49873...","[81971.77795004952, 95608.01000357303, 142093....",1.519804e+06,5.259745,17.349356,38.430002,-122.271529
4,"[71187.68931894137, 91325.47762727663, 126537....","[87.86079743893579, 112.648830751238, 158.8739...","[2.834219272223735, 4.02317252682993, 5.124965...","[75096.34961384782, 95950.40507552765, 132856....",1.522243e+06,5.380545,17.377201,38.459120,-121.560230
...,...,...,...,...,...,...,...,...,...
995,"[107265.182354745, 134276.019867511, 128014.07...","[124.1384581270522, 158.5507983603165, 156.865...","[4.004466391195231, 5.662528512868448, 5.06018...","[112428.6481571912, 140902.6655321975, 134651....",1.459818e+06,5.027627,16.664588,39.221826,-97.688818
996,"[107265.182354745, 134276.019867511, 128014.07...","[124.1384581270522, 158.5507983603165, 156.865...","[4.004466391195231, 5.662528512868448, 5.06018...","[112428.6481571912, 140902.6655321975, 134651....",1.459818e+06,5.027627,16.664588,39.320387,-97.763865
997,"[107265.182354745, 134276.019867511, 128014.07...","[124.1384581270522, 158.5507983603165, 156.865...","[4.004466391195231, 5.662528512868448, 5.06018...","[112428.6481571912, 140902.6655321975, 134651....",1.459818e+06,5.027627,16.664588,39.418948,-97.820999
998,"[92789.40788623082, 88502.69848567061, 147788....","[110.863837704466, 106.2788310789551, 185.1160...","[3.576252829176322, 3.795672538534109, 5.97148...","[97488.8026381655, 93219.84003819182, 155050.4...",1.487156e+06,5.235474,16.976670,39.517508,-112.266638


In [ ]:
# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(df1['longitude'], df1['latitude'])]
geo_df   = gpd.GeoDataFrame(df1, geometry=geometry)

In [ ]:
# Read the California county boundaries shapefile
ca_counties = gpd.read_file(data_path+ 'CA_counties/CA_Counties_TIGER2016.shp')

geo_df.crs  = "EPSG:4326" #Setting the crs of geo_df
#ca_counties = ca_counties.to_crs("EPSG:4326")
print('The coordinate reference system for this file:',ca_counties.crs,geo_df.crs)


# # Ensure that your data is in the same CRS as the shapefile
geo_df = geo_df.to_crs(ca_counties.crs)

# Plot the county boundaries
fig, ax = plt.subplots(figsize=(15, 15))
ca_counties.plot(ax=ax, color='white', edgecolor='black')

# Plot your data on top of the county boundaries
geo_df.plot(ax=ax, column='ac_annual', legend=True, markersize=50, cmap='OrRd', alpha=0.5)

# Add titles and labels as needed
plt.title("Annual AC Output in California by County")
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Optionally, adjust the plot limits to the extent of the shapefile or your data points
ax.set_xlim(ca_counties.bounds.minx.min(), ca_counties.bounds.maxx.max())
ax.set_ylim(ca_counties.bounds.miny.min(), ca_counties.bounds.maxy.max())

# Display the plot
plt.show()

In [ ]:
# Load US boundaries and plot
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#us = world[world.name == "United States of America"]
ca  = world[world.name=='California']

# Plot
fig, ax = plt.subplots(figsize=(15,10))
ca.plot(ax=ax, color='lightgray')
geo_df.plot(ax=ax, column='ac_annual', legend=True, markersize=50, cmap='OrRd')  # Use the column parameter
plt.title("Annual AC Output on California")
plt.show()

In [ ]:
# {
  # "inputs": {
  #   "api_key": "DEMO_KEY",
  #   "azimuth": "180",
  #   "system_capacity": "4",
  #   "losses": "14",
  #   "array_type": "1",
  #   "module_type": "0",
  #   "gcr": "0.4",
  #   "dc_ac_ratio": "1.2",
  #   "inv_eff": "96.0",
  #   "radius": "0",
  #   "dataset": "nsrdb",
  #   "tilt": "10",
  #   "address": "boulder, co",
  #   "soiling": [
  #     12.0,
  #     4.0,
  #     45.0,
  #     23.0,
  #     9.0,
  #     99.0,
  #     67.0,
  #     12.54,
  #     54.0,
  #     9.0,
  #     0.0,
  #     7.6
  #   ],
  #   "albedo": "0.3",
  #   "bifaciality": "0.7"
  # },
  # "errors": [],
  # "warnings": [],
  # "version": "8.0.0",
  # "ssc_info": {
  #   "version": 275,
  #   "build": "Linux 64 bit GNU/C++ Oct  4 2022 03:10:07",
  #   "module": "pvwattsv8"
  # },
  # "station_info": {
  #   "lat": 40.0099983215332,
  #   "lon": -105.2600021362305,
  #   "elev": 1635.640014648438,
  #   "tz": -7.0,
  #   "location": "149190",
  #   "city": "",
  #   "state": "Colorado",
  #   "solar_resource_file": "149190.csv",
  #   "weather_data_source": "NSRDB PSM V3 GOES tmy-2020 3.2.0"
  # }